In [ ]:
import pickle
import pandas as pd
import numpy as np

In [ ]:
with open('data/18.08-3.09.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data_dict = {}
idx = 0
for match in data:
    location = match['location']
    date = match['date']
    teams = []
    teams = match['teams']
    team_0 = teams[0]['team']
    country_0 = teams[0]['county']
    team_1 = teams[1]['team']
    country_1 = teams[1]['county']
    for game in match['maps']:
        picks_0 = game['picks'][0]
        picks_1 = game['picks'][1]
        bans_0 = game['bans'][0]
        bans_1 = game['bans'][1]
        winner = game['winner']
        data_dict[idx] = {
            'location': location,
            'team_0': team_0,
            'team_1': team_1,
            'country_0': country_0,
            'country_1': country_1,
            'picks_0': picks_0,
            'bans_0': bans_0,
            'picks_1': picks_1,
            'bans_1': bans_1,
            'winner': winner
        }
        idx += 1
        
df = pd.DataFrame.from_dict(data_dict, orient='index')

In [ ]:
df = df[df.picks_0.apply(lambda x: '' not in x)].reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
df_picks_0 = df['picks_0'].apply(pd.Series)
df_picks_0.columns = [f'pick_0_{x}' for x in range(1, 6)]

df_picks_1 = df['picks_1'].apply(pd.Series)
df_picks_1.columns = [f'pick_1_{x}' for x in range(1, 6)]


df_bans_0 = df['bans_0'].apply(pd.Series)
df_bans_0.columns = [f'ban_0_{x}' for x in range(1, 8)]


df_bans_1 = df['bans_1'].apply(pd.Series)
df_bans_1.columns = [f'ban_1_{x}' for x in range(1, 8)]

In [ ]:
df = pd.concat([df.drop(columns=['winner']), df_picks_0, df_picks_1, df_bans_0, df_bans_1, df['winner']], axis=1)

In [ ]:
df_heroes = pd.concat([df_picks_0, df_picks_1, df_bans_0, df_bans_1], axis=1)
all_heroes = set(df_heroes.stack().tolist())
all_heroes.remove('')
all_heroes = sorted(all_heroes)

del(df_heroes)

In [ ]:
class FewHotEncoder:
    
    def __init__(self, words):
        self.list_of_words = sorted(set(words))
        self.empty_array = np.zeros(len(self.list_of_words))
    
    def encode(self, words):
        encoded = self.empty_array.copy()
        for word in words:
            encoded[self.list_of_words.index(word)] = 1
        return encoded
            

In [ ]:
enc = FewHotEncoder(all_heroes)

In [ ]:
all_heroes = np.array(all_heroes)

In [ ]:
encoded_picks_0 = df.picks_0.apply(lambda x: enc.encode(x).astype(int))
encoded_picks_1 = df.picks_1.apply(lambda x: enc.encode(x).astype(int))

encoded_picks = encoded_picks_0 - encoded_picks_1
encoded_picks = list(encoded_picks.apply(list))
encoded_picks = pd.DataFrame(encoded_picks)

In [ ]:
len(encoded_picks)

In [ ]:
df = df.drop(columns=['picks_0', 'picks_1', 'bans_0', 'bans_1'])

In [ ]:
df = pd.concat([df, encoded_picks], axis=1)

In [ ]:
df.location.unique()

In [ ]:
df.head()

In [ ]:
df.country_0.unique()